# Basic useage of the DSMS-Python-SDK

Before you run this tutorial: make sure to have access to an DSMS-instance of your interest, that you have installed this package and that you have copied the needed variables such as the `DSMS_HOST_URL` and `DSMS_TOKEN` into an `.env`-file.

First of all, make let us import the needed classes and functions for this tutortial.

In [2]:
import os

from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [3]:
dsms = DSMS(env="../.env")

### 1: Introduction

We can see which kind of DSMS-object we own as a user:

We can investigate what a KItem needs in order to be created. KItems are entirely based on [`Pydantic`](https://docs.pydantic.dev/latest/)-Models (v2), hence the properties (in `Pydantic` called `Fields`) are automatically validated once we set them. 

The schema of the KItem itself is a JSON schema which is machine-readable and can be directly incorporated into [Swagger](https://swagger.io/tools/swagger-ui/)-supported APIs like e.g. [`FastAPI`](https://fastapi.tiangolo.com/).

We can investigate the KTypes defined in the remote instance:

In [4]:
for ktype in dsms.ktypes:
    print(ktype)

KTypes.Organization
KTypes.Expert
KTypes.App
KTypes.DatasetCatalog
KTypes.Dataset


### 2: Create KItems

We can make new KItems by simple class-initiation:

In [5]:
item = KItem(
    name="foo123",
    ktype_id=dsms.ktypes.Dataset,
    custom_properties={"foo": "bar"},
)

item

KItem(

	name = foo123, 

	id = 6630c022-5c65-4fe4-a109-58828aedf145, 

	ktype_id = KTypes.Dataset, 

	slug = foo123-6630c022, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [], 

	avatar_exists = False, 

	contacts = [], 

	created_at = None, 

	updated_at = None, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = {
		foo: bar
	}, 

	hdf5 = None
)

Remember: changes are only syncronized with the DSMS when you call the `commit`-method:

In [6]:
dsms.commit()
item.url

'https://stahldigital.materials-data.space/dataset/foo123-6630c022'

As we can see, the object we created before running the `commit`-method has automatically been updated, e.g. with the creation- and update-timestamp:

In [7]:
item

KItem(

	name = foo123, 

	id = 6630c022-5c65-4fe4-a109-58828aedf145, 

	ktype_id = dataset, 

	slug = foo123-6630c022, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [
		{
			user_id: 681614d0-9c59-4ee6-8dd5-409260ce0980
		}
	], 

	avatar_exists = False, 

	contacts = [], 

	created_at = 2024-04-17 23:30:44.313881, 

	updated_at = 2024-04-17 23:30:44.313881, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = {
		foo: bar
	}, 

	hdf5 = None
)

### 3. Update KItems

Now, we would like to update the properties of our KItem we created previously.

Depending on the schema of each property (see `KItem.model_schema_json()` in the **Introduction** of this tutorial), we can simply use the standard `list`-method as we know them from basic Python (e.g. for the `annotations`, `attachments`, `external_link`, etc). 


Other properties which are not `list`-like can be simply set by attribute-assignment (e.g. `name`, `slug`, `ktype_id`, etc).

In [8]:
# specify the path to any arbitrary file to be uploaded
file = os.path.join("..", "README.md")

item.name = "foobar"
item.custom_properties.foobar = "foobar"
item.attachments.append({"name": file})
item.annotations.append(
    {
        "iri": "www.example.org/foo",
        "name": "example class",
        "namespace": "www.example.org",
    }
)
item.external_links.append(
    {"url": "http://example.org", "label": "example link"}
)
item.contacts.append({"name": "foo", "email": "foo@bar.mail"})
item.affiliations.append({"name": "foobar team"})
item.user_groups.append({"name": "foogroup", "group_id": "123"})

Changes are sent to the DSMS through the `commit`-method again.

In [9]:
dsms.commit()

We can see now that e.g. the local system path of the attachment is changed to a simply file name, which means that the upload was successful. If not so, an error would have beem thrown during the `commit`.

Furthermore we can also download the file we uploaded again:

In [10]:
for file in item.attachments:
    download = file.download()

    print("\t\t\t Downloaded file:", file)
    print("|------------------------------------Beginning of file------------------------------------|")
    print(download)
    print("|---------------------------------------End of file---------------------------------------|")

			 Downloaded file: {
			name: README.md
		}
|------------------------------------Beginning of file------------------------------------|
# DSMS-SDK
Python SDK core-package for interacting with the Dataspace Management System (DSMS)


## Authors

[Matthias Büschelberger](mailto:matthias.bueschelberger@iwm.fraunhofer.de) (Fraunhofer Institute for Mechanics of Materials IWM)

[Yoav Nahshon](mailto:yoav.nahshon@iwm.fraunhofer.de) (Fraunhofer Institute for Mechanics of Materials IWM)

[Pablo De Andres](mailto:pablo.de.andres@iwm.fraunhofer.de) (Fraunhofer Institute for Mechanics of Materials IWM)

## License

This project is licensed under the BSD 3-Clause. See the LICENSE file for more information.

## Usage

The SDK provides a general Python interface to a remote DSMS deployment, allowing users to access, store and link data in a DSMS instance easily and safely. The package provides the following main capabilities:

- Managing Knowledge-Items (KItems), which are data instances of an expl

### 4: Delete KItems and their properties

We can also remove properties from the KItem without deleting the KItem itself.

For the `list`-like properties, we can use the standard `list`-methods from basic Python again (e.g. `pop`, `remove`, etc. or the `del`-operator).

For the other, non-`list`-like properties, we can simply use the attribute-assignment again.

When we only want single parts of the properties in the KItem, we can do it like this:

In [11]:
item.attachments.pop(0)
item.annotations.pop(0)
item.external_links.pop(0)
item.contacts.pop(0)
item.user_groups.pop(0)

{
			name: foogroup,
			group_id: 123
		}

However, we can also reset the entire property by setting it to e.g. an empty list again:

In [12]:
item.affiliations = []

See the changes:

In [13]:
item

KItem(

	name = foobar, 

	id = 6630c022-5c65-4fe4-a109-58828aedf145, 

	ktype_id = dataset, 

	slug = foo123-6630c022, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [
		{
			user_id: 681614d0-9c59-4ee6-8dd5-409260ce0980
		}
	], 

	avatar_exists = False, 

	contacts = [], 

	created_at = 2024-04-17 23:30:44.313881, 

	updated_at = 2024-04-17 23:30:57.022527, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = {
		foo: bar
	}, 

	hdf5 = None
)

Send the changes to the DSMS with the `commit`-method:

In [14]:
dsms.commit()

However, we can also delete the whole KItem from the DSMS by applying the `del`-operator to the `dsms`-object with the individual `KItem`-object:

In [15]:
del dsms[item]

Commit the changes:

In [16]:
dsms.commit()

### 5: Search for KItems

In the last unit of this tutorial, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also wnat to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`-attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [17]:
item = KItem(
    name="foo 1",
    ktype_id=dsms.ktypes.DatasetCatalog
)

item2 = KItem(
    name="foo 2",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item],
    annotations=[
        {
            "iri": "www.example.org/foo",
            "name": "foo",
            "namespace": "www.example.org",
        }
    ],
)
item3 = KItem(
    name="foo 3", 
    ktype_id=dsms.ktypes.Organization
)
item4 = KItem(
    name="foo 4",
    ktype_id=dsms.ktypes.Organization,
    annotations=[
        {
            "iri": "www.example.org/bar",
            "name": "bar",
            "namespace": "https://www.example.org",
        }
    ],
)

dsms.commit()

Now, we are apply to search for e.g. kitems of type `DatasetCatalog`:

In [18]:
dsms.search(ktypes=[dsms.ktypes.DatasetCatalog])

[SearchResult(hit=KItem(
 
 	name = csv_test_bulge, 
 
 	id = 2539dfde-abc9-4fdd-9204-026252ee952c, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = csvtestbulge, 
 
 	annotations = [], 
 
 	attachments = [
 		{
 			name: DX56_D_FZ2_WR00_43 1.txt
 		}, 
 		{
 			name: DX56_D_FZ2_WR00_43.csv
 		}
 	], 
 
 	linked_kitems = [
 		{
 			id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d,
 			source_id: 2539dfde-abc9-4fdd-9204-026252ee952c
 		}
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		{
 			user_id: c61e1c33-1658-4199-a809-ff693684cc70
 		}
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-02-15 13:05:49.418303, 
 
 	updated_at = 2024-02-15 13:05:49.418303, 
 
 	external_links = [], 
 
 	kitem_apps = [
 		{
 			kitem_app_id: 6,
 			executable: csv_tensile_test.ipynb,
 			title: csv_notebook,
 			description: Csv_notebook,
 			tags: None,
 			additional_properties: {triggerUponUpload: True, triggerUponUploadFileExtensions: ['.csv', '.txt']}
 		}
 	], 
 
 	summary = None, 


... and for all of type `Organization` and `DatasetCatalog`:

In [19]:
dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.DatasetCatalog])

[SearchResult(hit=KItem(
 
 	name = Fraunhofer, 
 
 	id = 6c695c8b-06ec-4df8-b840-d2dc0752e22d, 
 
 	ktype_id = organization, 
 
 	slug = fraunhofer, 
 
 	annotations = [
 		{
 			iri: http://www.oie.eu/ontology/characterization-method#ElectricalCharacterisationMethods,
 			name: ElectricalCharacterisationMethods,
 			namespace: method
 		}
 	], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		{
 			id: 2539dfde-abc9-4fdd-9204-026252ee952c,
 			source_id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d
 		}, 
 		{
 			id: b23df795-9441-4a0d-993a-aaeee53c4164,
 			source_id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d
 		}, 
 		{
 			id: 0f995978-9932-436a-9d76-961463457ed2,
 			source_id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d
 		}
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		{
 			user_id: 8d01f084-3604-4956-bd42-a09610e71d0e
 		}
 	], 
 
 	avatar_exists = True, 
 
 	contacts = [], 
 
 	created_at = 2024-02-13 15:20:23.000643, 
 
 	updated_at = 2024-02-13 15:20:23.000643, 
 
 	external_links = [], 
 

... or for all of type `DatasetCatalog` with `foo` in the name:

In [20]:
dsms.search(query="foo", ktypes=[dsms.ktypes.DatasetCatalog])

[SearchResult(hit=KItem(
 
 	name = foo 1, 
 
 	id = 8e3861e6-3479-411d-a460-addb91459da4, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = foo1-8e3861e6, 
 
 	annotations = [], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		{
 			id: dd54b41e-99fb-428e-a1f3-4812d0207e89,
 			source_id: 8e3861e6-3479-411d-a460-addb91459da4
 		}
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		{
 			user_id: 681614d0-9c59-4ee6-8dd5-409260ce0980
 		}
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-04-17 23:31:25.540343, 
 
 	updated_at = 2024-04-17 23:31:25.540343, 
 
 	external_links = [], 
 
 	kitem_apps = [], 
 
 	summary = None, 
 
 	user_groups = [], 
 
 	custom_properties = {
 		id: 8e3861e6-3479-411d-a460-addb91459da4, 
 		content: {}
 	}, 
 
 	hdf5 = None
 ), fuzzy=False)]

... and for all of type `Organization` with the annotation `www.example.org/foo`:

In [21]:
dsms.search(
        ktypes=[dsms.ktypes.Organization], annotations=["www.example.org/foo"]
    )

[SearchResult(hit=KItem(
 
 	name = Fraunhofer, 
 
 	id = 6c695c8b-06ec-4df8-b840-d2dc0752e22d, 
 
 	ktype_id = organization, 
 
 	slug = fraunhofer, 
 
 	annotations = [
 		{
 			iri: http://www.oie.eu/ontology/characterization-method#ElectricalCharacterisationMethods,
 			name: ElectricalCharacterisationMethods,
 			namespace: method
 		}
 	], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		{
 			id: 2539dfde-abc9-4fdd-9204-026252ee952c,
 			source_id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d
 		}, 
 		{
 			id: b23df795-9441-4a0d-993a-aaeee53c4164,
 			source_id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d
 		}, 
 		{
 			id: 0f995978-9932-436a-9d76-961463457ed2,
 			source_id: 6c695c8b-06ec-4df8-b840-d2dc0752e22d
 		}
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		{
 			user_id: 8d01f084-3604-4956-bd42-a09610e71d0e
 		}
 	], 
 
 	avatar_exists = True, 
 
 	contacts = [], 
 
 	created_at = 2024-02-13 15:20:23.000643, 
 
 	updated_at = 2024-02-13 15:20:23.000643, 
 
 	external_links = [], 
 

Clean up the DSMS from the tutortial:

In [22]:
del dsms[item]
del dsms[item2]
del dsms[item3]
del dsms[item4]

dsms.commit()


### 6. Apps

We can investigate which apps are available through JupyterLab:

In [23]:
dsms.apps

[App(filename='csv_bulgetest/csv_bulgetest.ipynb', basename='csv_bulgetest.ipynb', folder='csv_bulgetest'),
 App(filename='csv_tensile_test/csv_tensile_test.ipynb', basename='csv_tensile_test.ipynb', folder='csv_tensile_test'),
 App(filename='csv_tensile_test_f2/csv_tensile_test_f2.ipynb', basename='csv_tensile_test_f2.ipynb', folder='csv_tensile_test_f2'),
 App(filename='excel_component_test/excel_component_test.ipynb', basename='excel_component_test.ipynb', folder='excel_component_test'),
 App(filename='excel_nakajima_test/excel_nakajima_test.ipynb', basename='excel_nakajima_test.ipynb', folder='excel_nakajima_test'),
 App(filename='excel_notch_test/excel_notch_tensile_test.ipynb', basename='excel_notch_tensile_test.ipynb', folder='excel_notch_test'),
 App(filename='excel_tensile_test/excel_tensile_test.ipynb', basename='excel_tensile_test.ipynb', folder='excel_tensile_test'),
 App(filename='tensiletest-parameter-identification/tensiletest_parameter_identification.ipynb', basename='t

7. HDF5 

We are also able to upload dataframes or time series data and investigate them:

In [24]:
data = {"a": list(range(100)), "b": list(range(1,101))}


item = KItem(name="testdata123", ktype_id=dsms.ktypes.DatasetCatalog, hdf5=data)
dsms.commit()

print("Column-wise:")
for column in item.hdf5:
    print("column:", column.name, ",\n", "data:", column.get())

df = item.hdf5.to_df()
print("\nAs data frame:")
print(df)

new_df = df.drop(['a'], axis=1)
item.hdf5 = new_df

dsms.commit()

Column-wise:
column: a ,
 data: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0]
column: b ,
 data: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 

In [25]:
del dsms[item]

In [26]:
dsms.commit()